# Evaluating Summarization with TruLens

In this notebook, we will evaluate a summarization application based on [DialogSum dataset](https://github.com/cylnlp/dialogsum) using a broad set of available metrics from TruLens. These metrics break down into three categories.

1. Ground truth agreement: For these set of metrics, we will measure how similar the generated summary is to some human-created ground truth. We will use for different measures: BERT score, BLEU, ROUGE and a measure where an LLM is prompted to produce a similarity score.
2. Groundedness: Estimate if the generated summary can be traced back to parts of the original transcript both with LLM and NLI methods.
3. Comprehensivenss: Estimate if the generated summary contains all of the key points from the source text.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/use_cases/summarization_eval.ipynb)

In [1]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-'
os.environ['HUGGINGFACE_API_KEY'] = "hf_..."

### Dependencies
Let's first install the packages that this notebook depends on. Uncomment these linse to run.

In [ ]:
#!pip install trulens_eval bert_score evaluate absl-py rouge-score pandas tenacity

### Download and load data
Now we will download a portion of the DialogSum dataset from github.

In [5]:
import pandas as pd    

In [6]:
!wget -O dialogsum.dev.jsonl https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.dev.jsonl

--2024-05-14 17:11:32--  https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.dev.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471830 (461K) [text/plain]
Saving to: ‘dialogsum.dev.jsonl’

dialogsum.dev.jsonl 100%[===================>] 460.77K  --.-KB/s    in 0.03s   

2024-05-14 17:11:32 (15.7 MB/s) - ‘dialogsum.dev.jsonl’ saved [471830/471830]



In [7]:
file_path_dev = 'dialogsum.dev.jsonl'
dev_df = pd.read_json(path_or_buf=file_path_dev, lines=True)

Let's preview the data to make sure that the data was properly loaded

In [8]:
dev_df.head(10)

,fname,dialogue,summary,topic
0,dev_0,"#Person1#: Hello, how are you doing today?\n#P...",#Person2# has trouble breathing. The doctor as...,see a doctor
1,dev_1,#Person1#: Hey Jimmy. Let's go workout later t...,#Person1# invites Jimmy to go workout and pers...,do exercise
2,dev_2,#Person1#: I need to stop eating such unhealth...,#Person1# plans to stop eating unhealthy foods...,healthy foods
3,dev_3,#Person1#: Do you believe in UFOs?\n#Person2#:...,#Person2# believes in UFOs and can see them in...,UFOs and aliens
4,dev_4,#Person1#: Did you go to school today?\n#Perso...,#Person1# didn't go to school today. #Person2#...,go to school
5,dev_5,"#Person1#: Honey, I think you should quit smok...",#Person1# asks #Person2# to quit smoking for h...,quit smoking
6,dev_6,"#Person1#: Excuse me, Mr. White? I just need y...",Sherry reminds Mr. White to sign.,workplace conversation
7,dev_7,"#Person1#: Hey, Karen. Look like you got some ...",#Person1# asks Karen where Karen stayed and ho...,holidays
8,dev_8,#Person1#: How do you usually spend your leisu...,#Person1# asks about #Person2#'s hobbies. #Per...,hobby
9,dev_9,#Person1#: have you ever seen Bill Gate's home...,#Person1# and #Person2# talk about Bill Gate's...,dream home


## Create a simple summarization app and instrument it

We will create a simple summarization app based on the OpenAI ChatGPT model and instrument it for use with TruLens

In [9]:
from trulens_eval.tru_custom_app import instrument
from trulens_eval.tru_custom_app import TruCustomApp

/Users/gn03249822/opt/miniconda3/envs/trulens/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import openai

class DialogSummaryApp:
    
    @instrument
    def summarize(self, dialog):
        client = openai.OpenAI()
        summary = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                    {"role": "system", "content": """Summarize the given dialog into 1-2 sentences based on the following criteria: 
                     1. Convey only the most salient information; 
                     2. Be brief; 
                     3. Preserve important named entities within the conversation; 
                     4. Be written from an observer perspective; 
                     5. Be written in formal language. """},
                    {"role": "user", "content": dialog}
                ]
            ).choices[0].message.content
        return summary

## Initialize Database and view dashboard

In [ ]:
from trulens_eval import Tru
tru = Tru()
tru.reset_database()
# If you have a database you can connect to, use a URL. For example:
# tru = Tru(database_url="postgresql://hostname/database?user=username&password=password")

In [ ]:
tru.start_dashboard(force=True)

## Write feedback functions

We will now create the feedback functions that will evaluate the app. Remember that the criteria we were evaluating against were:
1. Ground truth agreement: For these set of metrics, we will measure how similar the generated summary is to some human-created ground truth. We will use for different measures: BERT score, BLEU, ROUGE and a measure where an LLM is prompted to produce a similarity score.
2. Groundedness: For this measure, we will estimate if the generated summary can be traced back to parts of the original transcript.

In [ ]:
from trulens_eval import Feedback, feedback
from trulens_eval.feedback import GroundTruthAgreement

We select the golden dataset based on dataset we downloaded

In [ ]:
golden_set = dev_df[['dialogue', 'summary']].rename(columns={'dialogue': 'query', 'summary': 'response'}).to_dict('records')

In [ ]:
from trulens_eval.feedback.provider import OpenAI, Huggingface
from trulens_eval import Select

provider = OpenAI()

ground_truth_collection = GroundTruthAgreement(golden_set, provider=provider)
f_groundtruth = Feedback(ground_truth_collection.agreement_measure, name = "Similarity (LLM)").on_input_output()
f_bert_score = Feedback(ground_truth_collection.bert_score).on_input_output()
f_bleu = Feedback(ground_truth_collection.bleu).on_input_output()
f_rouge = Feedback(ground_truth_collection.rouge).on_input_output()
# Groundedness between each context chunk and the response.
grounded_llm = feedback.Groundedness(groundedness_provider=provider)
grounded_nli = feedback.Groundedness(groundedness_provider=Huggingface())
f_groundedness_llm = (
    Feedback(grounded_llm.groundedness_measure_with_cot_reasons, name = "Groundedness - LLM Judge")
    .on(Select.RecordInput)
    .on(Select.RecordOutput)
    .aggregate(grounded_llm.grounded_statements_aggregator)
)
f_groundedness_nli = (
    Feedback(grounded_nli.groundedness_measure_with_nli, name = "Groundedness - NLI Judge")
    .on(Select.RecordInput)
    .on(Select.RecordOutput)
    .aggregate(grounded_nli.grounded_statements_aggregator)
)
f_comprehensiveness = (Feedback(provider.comprehensiveness_with_cot_reasons,
                            name = "Comprehensiveness")
                            .on(Select.RecordInput)
                            .on(Select.RecordOutput))



In [ ]:
provider.comprehensiveness_with_cot_reasons("the white house is white. obama is the president", "the white house is white. obama is the president")

## Create the app and wrap it

Now we are ready to wrap our summarization app with TruLens as a `TruCustomApp`. Now each time it will be called, TruLens will log inputs, outputs and any instrumented intermediate steps and evaluate them ith the feedback functions we created.

In [ ]:
app = DialogSummaryApp()
print(app.summarize(dev_df.dialogue[498]))

In [ ]:
tru_recorder = TruCustomApp(app, app_id='Summarize_v1',
                  feedbacks = [f_groundtruth,
                                f_groundedness_llm,
                                f_groundedness_nli,
                                f_comprehensiveness,
                                f_bert_score,
                                f_bleu,
                                f_rouge])

We can test a single run of the App as so. This should show up on the dashboard.

In [ ]:
with tru_recorder:
    app.summarize(dialog=dev_df.dialogue[498])

We'll make a lot of queries in a short amount of time, so we need tenacity to make sure that most of our requests eventually go through.

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def run_with_backoff(doc):
    return ta.with_record(app.summarize, dialog=doc)


In [ ]:
for pair in golden_set:
    llm_response = run_with_backoff(pair["query"])
    print(llm_response)

And that's it! This might take a few minutes to run, at the end of it, you can explore the dashboard to see how well your app does.

In [ ]:
tru.run_dashboard()